<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Introduction

This notebook updates the descriptive dataframe for a subsequent modeling or evaluating task. This descriptive dataframe serves as the leading input to the modeling pipeline and contains all information that is necessary to create the training and evaluating datasets.

The steps within this workflow are as follows:

1. accordingly to the initialized path the function loads the descriptive dataframe
2. the function splits the instances per SNR, per machine and per ID into training and testing and creates an additional column
3. the column is being added to the descriptive dataframe and saved back to the location

To use this notebook you will have to do the following steps:

1. define the path to the descriptive dataframe (path='....')
2. run all the cells after that

In [2]:
import pandas as pd
BASE_FOLDER = '../../'
%run -i ..\..\utility\modeling\JupyterLoad_modeling.py

load load_data
Load split_data
Load anomaly_detection_models
Load detection_pipe


In [11]:
diagrams = ['extdia_v0']
machines = ['pump', 'fan', 'slider', 'valve']
SNRs = ['6dB', 'min6dB']
IDs = ['00', '02', '04', '06']

for diagram in diagrams:
    for SNR in SNRs:
        for machine in machines:
            for ID in IDs:
                path = r'.\..\..\dataset\{}\{}{}{}_EDiaV0_pandaDisc.pkl'.format(diagram, 
                                                                                machine, 
                                                                                SNR, 
                                                                                ID)
                try:
                    tt_split(path)
                except:
                    print(path, 'probably not existent')

.\..\..\dataset\extdia_v0\pump6dB00_EDiaV0_pandaDisc.pkl --> Train test split already done, passed
.\..\..\dataset\extdia_v0\pump6dB02_EDiaV0_pandaDisc.pkl --> Train test split already done, passed
.\..\..\dataset\extdia_v0\pump6dB04_EDiaV0_pandaDisc.pkl --> Train test split already done, passed
.\..\..\dataset\extdia_v0\pump6dB06_EDiaV0_pandaDisc.pkl --> Done
.\..\..\dataset\extdia_v0\fan6dB00_EDiaV0_pandaDisc.pkl probably not existent
.\..\..\dataset\extdia_v0\fan6dB02_EDiaV0_pandaDisc.pkl probably not existent
.\..\..\dataset\extdia_v0\fan6dB04_EDiaV0_pandaDisc.pkl probably not existent
.\..\..\dataset\extdia_v0\fan6dB06_EDiaV0_pandaDisc.pkl probably not existent
.\..\..\dataset\extdia_v0\slider6dB00_EDiaV0_pandaDisc.pkl probably not existent
.\..\..\dataset\extdia_v0\slider6dB02_EDiaV0_pandaDisc.pkl probably not existent
.\..\..\dataset\extdia_v0\slider6dB04_EDiaV0_pandaDisc.pkl probably not existent
.\..\..\dataset\extdia_v0\slider6dB06_EDiaV0_pandaDisc.pkl probably not existent
.